In [75]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
iris = load_iris() 

from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer() 

from sklearn.datasets import load_digits
digits = load_digits() 

from sklearn.datasets import load_wine
wine = load_wine() 
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report

from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [62]:
def dataset(dataset):
    
    data = dataset
    x = data.data
    y = data.target
    z = data.target_names

    return x,y,z

In [63]:
def self_training(threshold,iteration,x_labled,x_unlabled,y_labled,y_unlabled,baseclasifier,y_train):
    
    true_label = y_train
    clf1 = baseclasifier
    x_labled = x_labled
    x_unlabled = x_unlabled
    y_labled = y_labled
    y_unlabled = y_unlabled
    threshold = threshold   #confidence
    i = iteration
    
    while len(x_unlabled) > 0  :  

        clf1.fit(x_labled, y_labled)
        y_pred1 = clf1.predict(x_unlabled)

        new_x_unlabled = np.empty((0,len(x_labled[0])),float)
        new_y_unlabled = []
        probability= np.empty((0,len(np.unique(y_labled))),float)
        
        for item in x_unlabled:

            p = clf1.predict_proba(np.matrix(item))
            p = p.astype(float)*100
            s = np.argmax(p) #the most probable label
            probability = np.append(probability,p,axis=0)
            if p.max() >= threshold:


                x_labled = np.append(x_labled,[item],axis=0)
                y_labled = np.append(y_labled,s)

            else:
                
                new_x_unlabled = np.append(new_x_unlabled,[item],axis=0)
                new_y_unlabled = np.append(new_y_unlabled,s)
        
            x_unlabled = new_x_unlabled
            y_unlabled = new_y_unlabled
            
        threshold = min(threshold,probability.max())
        print('x_unlabled:',len(x_unlabled), 'in %d iteration ' %i,' and ','threshold: ',threshold,'\n')
        
        i+=1
    return x_labled,y_labled,i

In [64]:
def testing_score(x_labled,y_labled,x_test,y_test,baseclasifier):
    
    x_lablel = x_labled
    y_labled = y_labled
    x_test = x_test
    y_test = y_test
    clf = baseclasifier
    clf.fit(x_labled,y_labled)

    y_pred = clf.predict(x_test)
    score = accuracy_score(y_test,y_pred)
    return score

In [65]:
#accuracy of labeled data
def transductive_score(x_train,y_train,new_x_labled,new_y_labled,target_names): 
    
    x_true = x_train
    y_true = y_train
    class_names = str((target_names))
    x_labled = new_x_labled
    y_pred = new_y_labled

    y_pred = np.reshape(y_pred,(len(y_pred),1))
    new_dataset = np.hstack((x_labled,y_pred))
    new_dataset_dF = pd.DataFrame(new_dataset)
    l = new_dataset_dF.columns.values.tolist()
    new_dataset_dF.sort_values(by=l,inplace=True)
    N=new_dataset_dF.values
    y= N[:,-1]
    
    y_train = np.reshape(y_train,(len(y_train),1))
    dataset = np.hstack((x_train,y_train))
    dataset_dF = pd.DataFrame(dataset)
    dataset_dF.sort_values(by=l,inplace=True)
    
    M=dataset_dF.values
    B= M[:,-1]

    score = classification_report(B,y)
    cm = confusion_matrix(B,y)

    return score,cm    

In [66]:
#choose classifier
def clf(clf,x_labled,y_labled,x_unlabled):
    
    clf = clf()
    x_lablel = x_labled
    y_labled = y_labled
    x_unlabled = x_unlabled

    clf.fit(x_labled,y_labled)
    y_pred = clf.predict(x_unlabled)
    
    return y_pred

In [67]:
def self_ensemble(x_labled, x_unlabled, y_labled, y_unlabled, clf1,clf2,clf3):
    
    clf1 = clf1
    clf2 = clf2
    clf3 = clf3
    x_labled = x_labled
    y_labled = y_labled
    x_unlabled = x_unlabled
    y_unlabled = y_unlabled
    
    flag = 0
#     while flag == 0 :  
    while len(x_unlabled) != 0:
        y_pred1 = clf(clf1,x_labled,y_labled,x_unlabled)
        y_pred2 = clf(clf2,x_labled,y_labled,x_unlabled)
        y_pred3 = clf(clf3,x_labled,y_labled,x_unlabled)
        
        m = np.zeros_like(y_pred1)
        for i in range(len(y_pred1)):
            if y_pred1[i] == y_pred2[i] == y_pred3[i]:
                m[i] += 1
                
        for i in range(len(y_pred1)):
            if y_pred1[i] == y_pred2[i] == y_pred3[i]:
                
                x_labled = np.append(x_labled,[x_unlabled[i]],axis=0)
                y_labled = np.append(y_labled, y_pred1[i]) 
    
                
        n = []
        for i in range(len(m)):
            if m[i] == 1:
                n = np.append(n,i)
                n = n.astype(int)
#         print(n)
        
        x_unlabled = np.delete(x_unlabled,n,axis=0) 
        y_unlabled = np.delete(y_unlabled,n,axis=0)
        print('Classifiers agree about label for ',len(n),'unlabled data')
    
        print(len(x_unlabled),"are still unlabled!\n")
        
        if(len(n) == 0):

            print("\nUsing VotingClassifier for ",len(x_unlabled),"unlabled data","\n")
            estimators = []
            model1 = clf1() #
            estimators.append(('MLP', model1))
            model2 = clf2() #DecisionTreeClassifier()
            estimators.append(('DTC', model2))
            model3 = clf3() #KNeighborsClassifier()
            estimators.append(('KNN', model3))

            # create the ensemble model
            ensemble = VotingClassifier(estimators)
            ensemble = ensemble.fit(x_labled, y_labled)
            pred_ensemble = ensemble.predict(x_unlabled)

            for i in range(len(pred_ensemble)):
                x_labled = np.append(x_labled,[x_unlabled[i]],axis=0)
                y_labled = np.append(y_labled, pred_ensemble[i]) 


            n = []
            for i in range(len(pred_ensemble)):
                n = np.append(n,i)
                n = n.astype(int)
            x_unlabled = np.delete(x_unlabled,n,axis=0) 
            y_unlabled = np.delete(y_unlabled,n,axis=0)    

            print('x_unlabled: ',len(x_unlabled),'\n All data are labled.\n')
    return x_labled,y_labled

In [69]:
def confusionmatrix(cm,targetnames):
    cm = cm
    l = targetnames
    plt.figure(figsize=(5,5))
    plt.imshow(cm,cmap=plt.cm.Blues,interpolation='nearest')
    plt.xticks(range(len(l)))
    plt.yticks(range(len(l)))
    plt.colorbar()
    plt.show()

In [70]:
def k_batch_self_ensemble(k,x_labled, x_unlabled, y_labled, y_unlabled,clf1,clf2,clf3):
   
    
    flag = 0
    while flag == 0 :  
    
    
        y_pred1 = clf(clf1,x_labled,y_labled,x_unlabled)
        y_pred2 = clf(clf2,x_labled,y_labled,x_unlabled)
        y_pred3 = clf(clf3,x_labled,y_labled,x_unlabled) 
    
        l=0 #add until k
        i = 0
        m = [] #common element
        while l < k and i<len(y_pred1):
            if y_pred1[i] == y_pred2[i] == y_pred3[i]:
                x_labled = np.append(x_labled,[x_unlabled[i]],axis=0)
                y_labled = np.append(y_labled, y_pred1[i]) 
                m = np.append(m,i)
                l +=1
                i +=1

            else:
                i +=1
        
        m = m.astype(int)
        x_unlabled = np.delete(x_unlabled,m,axis=0) 
        y_unlabled = np.delete(y_unlabled,m,axis=0)

        if(len(m) < k):
            flag = 1
     
    #Using of voting for the other unlabeld data
    
    if len(x_unlabled) > 0:
    
            print("\nUsing VotingClassifier for ",len(x_unlabled),"unlabled data","\n")
            estimators = []
            model1 = clf1() #
            estimators.append(('MLP', model1))
            model2 = clf2() #DecisionTreeClassifier()
            estimators.append(('DTC', model2))
            model3 = clf3() #KNeighborsClassifier()
            estimators.append(('KNN', model3))

            # create the ensemble model
            ensemble = VotingClassifier(estimators)
            ensemble = ensemble.fit(x_labled, y_labled)
            pred_ensemble = ensemble.predict(x_unlabled)

            for i in range(len(pred_ensemble)):
                x_labled = np.append(x_labled,[x_unlabled[i]],axis=0)
                y_labled = np.append(y_labled, pred_ensemble[i]) 


            n = []
            for i in range(len(pred_ensemble)):
                n = np.append(n,i)
                n = n.astype(int)
            x_unlabled = np.delete(x_unlabled,n,axis=0) 
            y_unlabled = np.delete(y_unlabled,n,axis=0)    

#             print('x_unlabled: ',len(x_unlabled),'\n All data are labled.\n')
            
    return x_labled,y_labled

In [80]:
def main_k_batch(dataset_name,baseclassifier,classifier1,classifier2,classifier3):
    
    clf = baseclassifier()
    clf1 = classifier1
    clf2 = classifier2
    clf3 = classifier3
    name = dataset_name
    
    x,y,target_names = dataset(name)
    
    x_train,x_test,y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0 , stratify=y)
    x_labled,x_unlabled,y_labled,y_unlabled = train_test_split(x_train, y_train, test_size=0.90, random_state=0 , stratify=y_train)
    
    
    
    super_score = testing_score(x_train,y_train,x_test,y_test,clf)
    semi_score = testing_score(x_labled, y_labled,x_test,y_test,clf)   #mohasebeye deghat ba 12 ta label dar
    
    
    
    print("\t Using self-training for labeling unlanled data: \n")
    new_x_labled, new_y_labled, iteration = self_training(70.0, 0, x_labled, x_unlabled, y_labled, y_unlabled, clf,y_train)
    self_trans_score,self_cm = transductive_score(x_train,y_train,new_x_labled,new_y_labled,target_names)
    self_score = testing_score(new_x_labled, new_y_labled,x_test,y_test,clf)
    print('confusion matrix of self-training \n',confusionmatrix(self_cm,target_names),'\n')
    print("Accuracy of self-training in labeling: \n",self_trans_score,'\n\n')
    
    print("\t Using ensemble-self-training for labling unlanled data: \n")
    ensemble_x_labled, ensemble_y_labled = self_ensemble(x_labled, x_unlabled, y_labled, y_unlabled,clf1,clf2,clf3)
    self_ensemble_score = testing_score(ensemble_x_labled,ensemble_y_labled,x_test,y_test,clf)
    ensemble_trans_score,ensemble_cm = transductive_score(x_train,y_train,ensemble_x_labled,ensemble_y_labled,target_names)
    print('confusion matrix of ensemble-self-training \n',confusionmatrix(ensemble_cm,target_names),'\n')
    
    print("Accuracy of ensemble-self-training in labling: \n",ensemble_trans_score,'\n\n')
    
    print("\t Using K_batch_self-ensemble for labeling unlanled data: \n")
    kb_ensemble_x_labled, kb_ensemble_y_labled = k_batch_self_ensemble(10, x_labled, x_unlabled, y_labled, y_unlabled,clf1,clf2,clf3)
    kb_self_ensemble_trans_score,kb_cm = transductive_score(x_train,y_train,kb_ensemble_x_labled,kb_ensemble_y_labled,target_names)
    kb_self_ensemble_score = testing_score(new_x_labled, new_y_labled,x_test,y_test,clf)
    print('confusion matrix of ensemble-self-training \n',confusionmatrix(kb_cm,target_names),'\n')
    print("Accuracy of K_batch_self-ensemble in labeling: \n",kb_self_ensemble_score,'\n\n')
    
    print("Accuracy supervised algorithm = %0.2f " %(super_score * 100) + "% \n")    
    print("Accuracy of semi-supervised(before labling) = %0.2f " %(semi_score * 100) + "% \n")
    print("Accuracy after labling by self-training = %0.2f " %(self_score* 100) + "% \n")
    print("Accuracy after labeling by self-ensemble = %0.2f " %(self_ensemble_score * 100) + "% \n")
    print("Accuracy after labeling by K_batch_self-ensemble = %0.2f " %(kb_self_ensemble_score * 100) + "% \n") 
    
    return super_score, semi_score, self_score, self_ensemble_score, kb_self_ensemble_score

In [81]:
################### save to excel ############################


d = pd.DataFrame(columns = col_names)
super_score_array, semi_score_array, self_score_array, self_ensemble_score_array, kb_self_ensemble_score_array = [], [], [], [], []
for i in range(10):
    super_score, semi_score, self_score, self_ensemble_score, kb_self_ensemble_score = main_k_batch(breast_cancer,KNeighborsClassifier,GaussianNB,tree.DecisionTreeClassifier,KNeighborsClassifier)
   
    super_score_array = np.append(super_score_array,super_score)
    semi_score_array = np.append(semi_score_array,semi_score)
    self_score_array = np.append(self_score_array,self_score)
    self_ensemble_score_array = np.append(self_ensemble_score_array,self_ensemble_score)
    kb_self_ensemble_score_array = np.append(kb_self_ensemble_score_array,kb_self_ensemble_score)
    
d = {'super':super_score_array, 'semi':semi_score_array,'self':self_score_array, 'self_ensemble':self_ensemble_score_array, 'kb_self_ensemble':kb_self_ensemble_score_array}
df1 = pd.DataFrame(data=d)

d = pd.DataFrame(columns = col_names)
super_score_array, semi_score_array, self_score_array, self_ensemble_score_array, kb_self_ensemble_score_array = [], [], [], [], []
for i in range(10):
    super_score, semi_score, self_score, self_ensemble_score, kb_self_ensemble_score = main_k_batch(breast_cancer,GaussianNB,GaussianNB,tree.DecisionTreeClassifier,KNeighborsClassifier)
   
    super_score_array = np.append(super_score_array,super_score)
    semi_score_array = np.append(semi_score_array,semi_score)
    self_score_array = np.append(self_score_array,self_score)
    self_ensemble_score_array = np.append(self_ensemble_score_array,self_ensemble_score)
    kb_self_ensemble_score_array = np.append(kb_self_ensemble_score_array,kb_self_ensemble_score)
    
d = {'super':super_score_array, 'semi':semi_score_array,'self':self_score_array, 'self_ensemble':self_ensemble_score_array, 'kb_self_ensemble':kb_self_ensemble_score_array}
df2 = pd.DataFrame(data=d)


d = pd.DataFrame(columns = col_names)
super_score_array, semi_score_array, self_score_array, self_ensemble_score_array, kb_self_ensemble_score_array = [], [], [], [], []
for i in range(10):
    super_score, semi_score, self_score, self_ensemble_score, kb_self_ensemble_score = main_k_batch(breast_cancer,tree.DecisionTreeClassifier,GaussianNB,tree.DecisionTreeClassifier,KNeighborsClassifier)
   
    super_score_array = np.append(super_score_array,super_score)
    semi_score_array = np.append(semi_score_array,semi_score)
    self_score_array = np.append(self_score_array,self_score)
    self_ensemble_score_array = np.append(self_ensemble_score_array,self_ensemble_score)
    kb_self_ensemble_score_array = np.append(kb_self_ensemble_score_array,kb_self_ensemble_score)
    
d = {'super':super_score_array, 'semi':semi_score_array,'self':self_score_array, 'self_ensemble':self_ensemble_score_array, 'kb_self_ensemble':kb_self_ensemble_score_array}
df3 = pd.DataFrame(data=d)

with pd.ExcelWriter('output_iris2.xlsx') as writer:  # doctest: +SKIP
    df1.to_excel(writer, sheet_name='KNN')
    df2.to_excel(writer, sheet_name='NB')
    df3.to_excel(writer, sheet_name='DT')

x_unlabled: 51 in 0 iteration   and  threshold:  70.0 

x_unlabled: 7 in 1 iteration   and  threshold:  70.0 

x_unlabled: 0 in 2 iteration   and  threshold:  70.0 

Classifiers agree about label for  356 unlabled data
54 are still unlabled!

Classifiers agree about label for  19 unlabled data
35 are still unlabled!

Classifiers agree about label for  1 unlabled data
34 are still unlabled!

Classifiers agree about label for  1 unlabled data
33 are still unlabled!

Classifiers agree about label for  0 unlabled data
33 are still unlabled!


Using VotingClassifier for  33 unlabled data 

x_unlabled:  0 
 All data are labled.



C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



Using VotingClassifier for  31 unlabled data 

x_unlabled:  0 
 All data are labled.



C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


x_unlabled: 51 in 0 iteration   and  threshold:  70.0 

x_unlabled: 7 in 1 iteration   and  threshold:  70.0 

x_unlabled: 0 in 2 iteration   and  threshold:  70.0 

Classifiers agree about label for  358 unlabled data
52 are still unlabled!

Classifiers agree about label for  17 unlabled data
35 are still unlabled!

Classifiers agree about label for  3 unlabled data
32 are still unlabled!

Classifiers agree about label for  0 unlabled data
32 are still unlabled!


Using VotingClassifier for  32 unlabled data 

x_unlabled:  0 
 All data are labled.



...

C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


x_unlabled: 0 in 0 iteration   and  threshold:  70.0 

Classifiers agree about label for  352 unlabled data
58 are still unlabled!

Classifiers agree about label for  15 unlabled data
43 are still unlabled!

Classifiers agree about label for  0 unlabled data
43 are still unlabled!


Using VotingClassifier for  43 unlabled data 

x_unlabled:  0 
 All data are labled.



C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



Using VotingClassifier for  31 unlabled data 

x_unlabled:  0 
 All data are labled.



C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
